# Player Session Insights
* Use jupyter notebook and spark to support your answers.
* How many sessions are in the dataset?
* How many sessions are completed per country?
* How many sessions are completed per player?
* What is the country with more sessions started during 2018?
* Plot the player sessions completed per country

## Gather the dataset
https://cdn.unityads.unity3d.com/assignments/assignment_data.jsonl.bz2 (30-40 MB)

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [7]:
# I downloaded the json file
df = spark.read.json('assignment_data.jsonl.bz2')

In [19]:
# examine basic structure
if not df.is_cached:
    df.cache()
df.printSchema()
print(df.head(2))
df.describe().show()

root
 |-- country: string (nullable = true)
 |-- event: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- ts: string (nullable = true)

[Row(country='PK', event='start', player_id='d6313e1fb7d247a6a034e2aadc30ab3f', session_id='674606b1-2270-4285-928f-eef4a6b90a60', ts='2016-11-22T20:40:50'), Row(country=None, event='end', player_id='20ac16ebb30a477087c3c7501b1fce73', session_id='16ca9d01-d240-4527-9f8f-00ef6cddb1d4', ts='2016-11-18T06:24:50')]
+-------+-------+-------+--------------------+--------------------+-------------------+
|summary|country|  event|           player_id|          session_id|                 ts|
+-------+-------+-------+--------------------+--------------------+-------------------+
|  count| 500584|1001169|             1001169|             1001169|            1001169|
|   mean|   null|   null|                null|                null|               null|
| stddev|   null|   null|                null|  

## Getting insight
* How many sessions are in the dataset? 1001169 sessions

In [18]:
df.count()

1001169

* How many sessions are completed per country?

In [118]:
# use sql, create a sql table from the dataframe
df.createOrReplaceTempView("df_table")
spark.sql("""
    SELECT COUNT(start.session_id), start.country
    FROM df_table start
    JOIN df_table end
        ON start.event <> end.event
        AND start.session_id = end.session_id
        AND start.ts < end.ts
    GROUP BY start.country
    ORDER BY COUNT(start.session_id) DESC
""").show()

+-----------------+-------+
|count(session_id)|country|
+-----------------+-------+
|             2839|     IT|
|             2672|     SH|
|             2670|     AZ|
|             2655|     AT|
|             2616|     WF|
|             2602|     VA|
|             2584|     HU|
|             2561|     IL|
|             2549|     RO|
|             2535|     KR|
|             2532|     SR|
|             2521|     TO|
|             2515|     MD|
|             2475|     LS|
|             2461|     NO|
|             2440|     DE|
|             2433|     TF|
|             2431|     EG|
|             2422|     CD|
|             2413|     GS|
+-----------------+-------+
only showing top 20 rows



* How many sessions are completed per player?

In [ ]:
spark.sql("""
    SELECT start.session_id, start.player_id
    FROM df_table start
    JOIN df_table end
        ON start.session_id = end.session_id
    WHERE start.ts < end.ts
        AND start.event <> end.event
""").groupBy(df.player_id).count().show()

* What is the country with more sessions started during 2018?